In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Configure gmaps to use Google API key.
gmaps.configure(api_key=g_key)

### Use Google API to Create Heatmaps

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()
# city_data_df.dtypes

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Tiksi,RU,2022-08-05 18:01:26,71.6872,128.8694,40.82,95,83,3.96
1,1,Jamestown,US,2022-08-05 17:56:53,42.0970,-79.2353,86.52,60,40,5.75
2,2,Constitucion,CL,2022-08-05 18:01:27,-35.3333,-72.4167,56.41,72,100,3.24
3,3,Homer,US,2022-08-05 18:01:27,59.6425,-151.5483,53.49,95,100,11.01
4,4,Maumere,ID,2022-08-05 18:01:28,-8.6199,122.2111,73.22,75,49,7.52


#### Temperature Heatmap

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

#### Humidity Heatmap

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

#### Cloudiness Heatmap

In [6]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
cloudiness = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=cloudiness, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

#### Wind Speed Heatmap

In [7]:
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Vacation Criteria
A feature on the app that allows customers to search for locations they want to travel based on their temperature preferences.

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)
# preferred_cities_df.isnull().sum()
#Showing under 200 cities for plotting visibility

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Jamestown,US,2022-08-05 17:56:53,42.0970,-79.2353,86.52,60,40,5.75
13,13,Atuona,PF,2022-08-05 18:01:30,-9.8000,-139.0333,76.82,71,74,15.23
14,14,Itarema,BR,2022-08-05 18:01:31,-2.9248,-39.9167,84.56,60,4,17.16
15,15,Kahului,US,2022-08-05 18:01:31,20.8947,-156.4700,81.84,82,7,1.01
17,17,Xingcheng,CN,2022-08-05 18:01:32,40.6167,120.7167,79.21,91,13,2.04
24,24,Trat,TH,2022-08-05 18:01:35,12.5000,102.5000,79.88,96,100,4.59
28,28,Tabou,CI,2022-08-05 18:01:36,4.4230,-7.3528,76.17,72,100,7.31
34,34,Albany,US,2022-08-05 18:01:38,42.6001,-73.9662,89.02,64,87,1.01
36,36,Vianopolis,BR,2022-08-05 18:01:39,-16.7419,-48.5164,85.55,18,48,1.97
44,44,Saint George,US,2022-08-05 18:02:41,37.1041,-113.5841,89.64,47,40,4.61


### Hotel Search

In [10]:
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Jamestown,US,86.52,42.0970,-79.2353,
13,Atuona,PF,76.82,-9.8000,-139.0333,
14,Itarema,BR,84.56,-2.9248,-39.9167,
15,Kahului,US,81.84,20.8947,-156.4700,
17,Xingcheng,CN,79.21,40.6167,120.7167,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


##### Hotel Pop-up markers

In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))